### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import os

# File to Load (Remember to Change These)
file_to_load = os.path.join("purchase_data.csv")

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [2]:
# Used unique on the SN column to get number of players
total_players_unique = len(purchase_data['SN'].unique())
total_players = pd.DataFrame({"Total Players":[total_players_unique]})
total_players

,Total Players
0,576


## Purchasing Analysis (Total)

In [3]:
#Unique Items Column
items = purchase_data['Item Name'].unique()
unique_items = len(list(items))

#Average Price column
average_price = purchase_data["Price"].mean()

#Number of purchases column
num_purch = len(list(purchase_data["Price"]))

#Total Revenue column
total_rev = purchase_data["Price"].sum(0)

#Create DataFrame with aquired data
purchasing_analysis = pd.DataFrame({"Number of Unique Items":[unique_items],"Average Price":[average_price],
                                 " Number of Purhcases":[num_purch], "Total Revenue":[total_rev]})

#Format columns 2 and 4 then display resulting table
purchasing_analysis["Average Price"] = purchasing_analysis["Average Price"].map("${:.2f}".format)
purchasing_analysis["Total Revenue"] = purchasing_analysis["Total Revenue"].map("${:.2f}".format)
purchasing_analysis


,Number of Unique Items,Average Price,Number of Purhcases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
# get unique names and corresponding gender data 
gender_unique = purchase_data[['SN','Gender']]

#Output from unique() is array, so used len to find counts
num_unique_males = len(gender_unique[gender_unique['Gender'] == 'Male']['SN'].unique())
num_unique_females = len(gender_unique[gender_unique['Gender'] == 'Female']['SN'].unique())
num_unique_NA = len(gender_unique[gender_unique['Gender'] == 'Other / Non-Disclosed']['SN'].unique())

# Percentage of each gender 
total_people= sum([num_unique_males,num_unique_females,num_unique_NA])

male_count_percentage = num_unique_males/total_people
female_count_percetnage = num_unique_females/total_people
none_count_percentage = num_unique_NA/total_people

# make the data frame and display
gender_demographics = pd.DataFrame({"Total Count":[num_unique_males,num_unique_females,num_unique_NA],"Percentage of Players":
[male_count_percentage, female_count_percetnage, none_count_percentage]}, index = ["Male", "Female", "N/A"])

gender_demographics

,Total Count,Percentage of Players
Male,484,0.840278
Female,81,0.140625
N/A,11,0.019097



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
#Get total gender purchases without using unique 
male  = purchase_data.loc[purchase_data['Gender'] == 'Male']
female  = purchase_data.loc[purchase_data['Gender'] == 'Female']
none  = purchase_data.loc[purchase_data['Gender'] == 'Other / Non-Disclosed']

#Purchase Count
male_count = male.shape[0]
female_count = female.shape[0]
none_count = none.shape[0]

# Average Purchase Price
average_purchase_male  = male['Price'].mean()
average_purchase_female = female['Price'].mean()
average_purchase_NA = none["Price"].mean()

# Total Purchase Value 
total_purchase_male  = male['Price'].sum()
total_purchase_female = female['Price'].sum()
total_purchase_NA = none["Price"].sum()

# Average Total Purchase Per Person (Using Unique Data in previous Problem)
per_male = total_purchase_male/num_unique_males
per_female = total_purchase_female/num_unique_females
per_NA = total_purchase_NA/num_unique_NA

gender_purchases = pd.DataFrame({"Purcahse Count":[male_count, female_count, none_count],
"Average Purchase Price":[average_purchase_male, average_purchase_female, average_purchase_NA], 
"Total Purchase Value": [total_purchase_male,total_purchase_female,total_purchase_NA],
"Average Total Purchase Per Person": [per_male, per_female, per_NA]}, index = ["Male", "Female", 'Other / Non-Disclosed'])

gender_purchases

,Purcahse Count,Average Purchase Price,Total Purchase Value,Average Total Purchase Per Person
Male,652,3.017853,1967.64,4.065372
Female,113,3.203009,361.94,4.468395
Other / Non-Disclosed,15,3.346000,50.19,4.562727


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
#extract continious age data to turn into categorical data 
age_demo = purchase_data['Age']

# base on n (#bins desired),max, and min, to not hard code bins
n=8
min = age_demo.min()
max = age_demo.max()
steps = int((round(((max-min)/n))))

#set up bins and bin names
#bins = [0,10,14,19,24,29,34,39,45]
bins = [0,9] + [x for x in range(14, max-1,steps)] + [max]
names = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

#Use cut to split up data and get value counts, then add to the rest of the data frame 
age_demo = pd.cut(age_demo,bins = bins,labels = names, include_lowest=True )
age_counts = age_demo.value_counts()
purchase_data['Age Group'] = age_demo

# create new data frame with indix column as age categories in order to use loc
df = purchase_data.set_index(age_demo)

#Build and display the data frame 
p_c = []
mean = []
sm = []
avg_per = []
for age in names:
    p_c.append(len(df['Price'].loc[age]))
    mean.append(float(df["Price"].loc[age].mean()))
    sm.append(float(df['Price'].loc[age].sum()))

    unique_age_count = len(df['SN'].loc[age].unique())
    avg_per.append(float(df["Price"].loc[age].sum())/unique_age_count)

age_df = pd.DataFrame({"Purchase Count":p_c,"Average Purchase Price":mean, "Total Purchase Value":sm,"Average Total Purchase Per Person":avg_per}, index = names)
age_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase Per Person
<10,23,3.353478,77.13,4.537059
10-14,28,2.956429,82.78,3.762727
15-19,136,3.035956,412.89,3.858785
20-24,365,3.052219,1114.06,4.318062
25-29,101,2.900990,293.00,3.805195
30-34,73,2.931507,214.00,4.115385
35-39,41,3.601707,147.67,4.763548
40+,13,2.941538,38.24,3.186667


## Age Demographics (Percentages)

In [7]:
unique_ages_percents = []

#gather data for percentages data frame
for age in names:
    unique_ages_percents.append(len(df['SN'].loc[age].unique()))

# Build DataFrame and create percentage column based on gathered data, then display 
percent_ages_df = pd.DataFrame({"Total Count": unique_ages_percents},index = names)
percent_ages_df['Percentage of Players'] = [age/sum(unique_ages_percents) for age in unique_ages_percents]
percent_ages_df

,Total Count,Percentage of Players
<10,17,0.029514
10-14,22,0.038194
15-19,107,0.185764
20-24,258,0.447917
25-29,77,0.133681
30-34,52,0.090278
35-39,31,0.053819
40+,12,0.020833


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
# Group by name, then use aggregate to run all three required functions. Then extract the price section
top_spenders = purchase_data.groupby(['SN']).agg(['count','mean', 'sum'])['Price']

# Build,sort by 'Total Purchase Value', and display DataFrame 
top_spenders = top_spenders.rename(columns = {"count": "Purchase Count","mean": "Average Purchase Price", "sum": "Total Purchase Value"})
top_spenders = top_spenders.sort_values(by  = ['Total Purchase Value'], ascending = False)
top_spenders.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.792000,18.96
Idastidru52,4,3.862500,15.45
Chamjask73,3,4.610000,13.83
Iral74,4,3.405000,13.62
Iskadarya95,3,4.366667,13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
#Get Columns for purchase count and total purchase value
gp_items = purchase_data.groupby(['Item ID','Item Name']).agg(['count','sum'])
gp_items = gp_items['Price']

# rename columns 
gp_items = gp_items.rename(columns = {'count': "Purchase Count", 'sum': 'Total Purchase Value'})

#add item price column 
gp_items['Item Price'] = gp_items['Total Purchase Value']/gp_items['Purchase Count']

#sort DataFrame
gp_items.sort_values(by = ['Purchase Count'], ascending = False)

,,Purchase Count,Total Purchase Value,Item Price
Item ID,Item Name,,,
92,Final Critic,13,59.99,4.614615
178,"Oathbreaker, Last Hope of the Breaking Storm",12,50.76,4.230000
145,Fiery Glass Crusader,9,41.22,4.580000
132,Persuasion,9,28.99,3.221111
108,"Extraction, Quickblade Of Trembling Hands",9,31.77,3.530000
...,...,...,...,...
42,The Decapitator,1,1.75,1.750000
51,Endbringer,1,4.66,4.660000
118,"Ghost Reaver, Longsword of Magic",1,2.17,2.170000


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
#Sort the above data frame 
gp_items_descending = gp_items.sort_values(by = ['Total Purchase Value'], ascending = False)
gp_items_descending.head()

,,Purchase Count,Total Purchase Value,Item Price
Item ID,Item Name,,,
92,Final Critic,13,59.99,4.614615
178,"Oathbreaker, Last Hope of the Breaking Storm",12,50.76,4.230000
82,Nirvana,9,44.10,4.900000
145,Fiery Glass Crusader,9,41.22,4.580000
103,Singed Scalpel,8,34.80,4.350000
